<a href="https://colab.research.google.com/github/ghkdwnwsdud/ksebProject/blob/main/modules/AIcaster_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##타석
#타석 여부 -모델 o
#피칭 여부 -모델 o
#스윙 여부 -모델 o

##주루
#이미지 캡셔닝 데이터 - 모델 제작중


#화면 전환 - 모델 o


## 상황판별
#볼카운트 데이터 -모델 제작중
#시간 데이터 - o
#점수 여부 - o / 프레임 잘라서 ocr
#진루 여부 - 제작중


In [ ]:
def preprocessing_frame(path):
    pass

In [ ]:
def frame_change(frame):
    pass

In [ ]:
def score_ocr(frame):
    pass

In [ ]:
def pitcher_hitter_detecting(frame):
    pass

In [ ]:
def swing_detecting(frame):
    pass

In [ ]:
def pitching_detecting(frame):
    pass

In [ ]:
def batter_box_detecting(frame):
    pass

In [ ]:
def count_detecting(frame):
    pass

In [ ]:
def image_captioning(frame):
    pass

In [ ]:
def call_api(tag):
    pass

In [ ]:
##w전역변수들
pre_st_count= 0
pre_b_count = 0
pre_out_count = 0
st_count = 0
b_count = 0
out_count = 0
time = 0
swing_count = -1
pitching_count = -1
flag = 0
state = None
count_flag
state_time = 0


In [ ]:
##인풋 프레임  // 시간 time을 고려해야함.
##태그 업데이트
tag = {'타석' : , '피칭' : , '스윙' :, '판정' : , '아웃_카운트' :, '볼_카운트' : , '스트_카운트' :, '주루': , 'time': , 'state' :, 'flag' :, '점수' :}

image_path = ""
frame = preprocessing(image_path)

#카운트 태그 업데이트
pre_st_count, pre_b_count, pre_out_count = tag['볼_카운트'], tag['스트_카운트'], tag['아웃_카운트']
if count_detecting(frame) != 0:
    tag['볼_카운트'], tag['스트_카운트'], tag['아웃_카운트'] = count_detecting(frame)
if (pre_st_count != tag['스트_카운트']) or (pre_b_count != tag['볼_카운트']) or (pre_out_count != tag['아웃_카운트']):
    tag['flag'] = 1

#점수 업데이트
tag['점수'] = score_ocr(frame)

if better_box_detecting(frame) == True: # 타석

    #피칭 태그 업데이트
    pitcher_box, hitter_box = pitcher_hitter_detecting(frame)
    pitching_detected = pitching_detecting(pitcher_box)
    if tag['피칭'] != pitching_detected:
        if pitching_count == -1:
            pitching_count = 3
        elif pitching_count == 0:
            tag['피칭'] = pitching_detected #피칭
        else :
            pitching_count -= 1
    else: #피칭X
        if pitching_count != -1:
            pitching_count = -1

    #스윙 태그 업데이트
    swing_detected = swing_detecting(hitter_box)
    if tag['스윙'] != swing_detected: #
        if swing_count == -1:
            swing_count = 3
        elif swing_count == 0:
            tag['스윙'] = swing_detected #스윙
        else :
            swing_count -= 1
    else: #스윙X
        if swing_count != -1:
            swing_count = -1
else : #주루 플레이



In [ ]:
##인풋 프레임  // 시간 time을 고려해야함.

#태그정보들을 바탕으로 state를 유지하거나, 변환한다.

prestate = state

if (state == None) and (tag['피칭'] == True): #피칭
    state = pitching_state
    state_time = 4

if (state == pitching_state) and (tag['스윙'] == True): #스윙
    state = swing_state
    state_time = 5

if (prestate != state) and (state_time == 0):
    state = None
    time = 0
elif (prestate == state) and (state_time == 0):
    pass
else :
    state_time -= 1

#flag가 업데이트 되면, api를 작동시킨다.
if (state == pitching_state) and (tag['flag'] == True): #noswing / 볼 스트라이크 판정 가능
    tag['flag'] = false
elif (state == swing_state) and (tag['flag'] == True): #swing / 스트라이크 아웃, 안타, 파울 판정 가능
    tag['flag'] = false
else :
    pass

#타석이 아닌 일반 주루플레이가 일정 시간 지속되면 주루플레이에 대한 api를 작동시킨다.
if (state = None) and time > 2 :
    text_gen = image_captioning(frame)
    time = 0



In [ ]:
#특정부분 색깔 인식
import cv2
import numpy as np

def is_green(image, top_left, bottom_right, green_threshold=0.5):
    """
    이미지의 특정 부분이 초록색인지 아닌지를 판별합니다.

    Parameters:
    image (numpy.ndarray): 입력 이미지 (BGR 형식)
    top_left (tuple): 잘라낼 영역의 왼쪽 상단 좌표 (x, y)
    bottom_right (tuple): 잘라낼 영역의 오른쪽 하단 좌표 (x, y)
    green_threshold (float): 초록색으로 간주할 픽셀의 비율 임계값

    Returns:
    bool: 특정 부분이 초록색이면 True, 아니면 False
    """
    # 지정된 영역 자르기
    cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

    # BGR 이미지를 HSV 이미지로 변환
    hsv_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

    # 초록색의 HSV 범위 정의
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])

    # 지정된 HSV 범위 내에 있는지 여부를 판별하는 마스크 생성
    mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # 마스크에서 초록색 픽셀의 비율 계산
    green_ratio = np.count_nonzero(mask) / mask.size

    # 초록색 픽셀이 일정 비율 이상이면 True 반환
    return green_ratio > green_threshold

# 예제 사용법
if __name__ == "__main__":
    # 이미지 읽기 (BGR 형식)
    image = cv2.imread('your_image_path.jpg')

    # 좌표 설정 (예: (50, 50)에서 (150, 150) 영역)
    top_left = (50, 50)
    bottom_right = (150, 150)

    # 초록색 판별 함수 호출
    is_green_area = is_green(image, top_left, bottom_right, green_threshold=0.5)

    # 결과 출력
    print("해당 영역이 초록색입니까?", is_green_area)

In [ ]:
#타입캐스트 api = __plt7qqFjo3YMYsqgtFX3gBGJ4cPhiL3zmtNVDYQVV62

import requests
import json

url = "https://typecast.ai/api/speak"

payload = json.dumps({
  "actor_id": "60228e8775ecc9e7ab8ef334",
  "text": "",
  "lang": "auto",
  "tempo": 1,
  "volume": 100,
  "pitch": 0,
  "xapi_hd": True,
  "max_seconds": 60,
  "model_version": "latest",
  "xapi_audio_format": "wav"
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer __pl....VV62'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
def frame_change(frame):
    pass

In [ ]:
def score_ocr(frame):
    pass

In [ ]:
def pitcher_hitter_detecting(frame):
    pass

In [ ]:
def swing_detecting(frame):
    pass

In [ ]:
def pitching_detecting(frame):
    pass

In [ ]:
def batter_box_detecting(frame):
    pass

In [ ]:
def count_detecting(frame):
    pass